In [170]:
# !pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /private/var/folders/73/dfnlcqg57w7btn76pt1r17cm0000gp/T/pip-install-05lpeeup/twint_99fbac0bb6064cd687fe5d0328094cf6
  Running command git clone -q https://github.com/twintproject/twint.git /private/var/folders/73/dfnlcqg57w7btn76pt1r17cm0000gp/T/pip-install-05lpeeup/twint_99fbac0bb6064cd687fe5d0328094cf6
  Running command git checkout -q origin/master
  Resolved https://github.com/twintproject/twint.git to commit origin/master


In [311]:
import pandas as pd
import twint
import re
import requests
import nest_asyncio
from textblob import TextBlob
nest_asyncio.apply()
import seaborn as sns
import matplotlib.pyplot as plt

In [312]:
candidate_name={'Peter Obi':'Obidients',
    'Bola Ahmed Tinubu':'BAT',
    'Atiku Abubakar':'Atiku'}

In [313]:
def twintConfig(search_string):
    c = twint.Config()
    c.Search = search_string[1]
    c.Limit=500
    c.Lang = 'en'
    c.Pandas = True
    c.Pandas_au = True          
    c.Pandas_clean=True
    c.Since='2022-01-01'
    c.Until='2022-12-31'
    twint.run.Search(c)

In [314]:
def Run_Twint(search_vals):
    
    #set empty dataframe for join
    out_df= pd.DataFrame()
    
    for candidate in search_vals.items():
        print ("running for search item: "+candidate[0]+"\n")
        print ("Search string: "+candidate[1]+"\n")
        
        #run twint
        twintConfig(candidate)
        
        #get dataframe
        df_all = twint.storage.panda.Tweets_df
        
        #join Dataframes and create 'Bank' column
        df_all["Candidate"]= candidate[0]
        out_df = pd.concat([out_df,df_all])
        
    return out_df

In [ ]:
df_all = Run_Twint(candidate_name)

running for search item: Peter Obi

Search string: Obidients

1605171800874389504 2022-12-20 13:02:34 +0100 <AkwaObidients> @PeterObi  At the Akwa Ibom Traditional Council Chamber  https://t.co/CtYupdve8O
1605171547622219777 2022-12-20 13:01:34 +0100 <OchukoEmeje> TO LET Partly furnished 3Bedroom flat in serene &amp; secure neighborhood, Okupe Estate,Maryland  2.5m(per annum) @TrafficChiefNG @Gidi_Traffic #LagosProperties #RealEstate #AkwaIbomWelcomesObiDatti #Obidients  https://t.co/ctSsgbf1AQ
1605171405309366277 2022-12-20 13:01:00 +0100 <Deus_Ugwoke> @chude__ Confirm nightmare. May the Almighty God continue to guide him &amp; all Obidients. Aso rock is our destination &amp; we'll get there by His grace.
1605171221330640897 2022-12-20 13:00:16 +0100 <Chris07249658> @OladejoBilewu PO and LP are suddenly corrupt because Okupe received cash and sinned for the PDP. Talk your mind bro. Closet PDAPC people like you probably still plenty dey answer Obidients.
1605171191395876866 2022-12-20 

1605161272735313922 2022-12-20 12:20:44 +0100 <bestbehavi_or> @AfiaUfot My people 💯
1605161193781575681 2022-12-20 12:20:25 +0100 <ElephantofA> @YeleSowore @doyinokupe I keep telling obidients. Life is about doing the right thing , about standing on the path of truth. Not following the crowd. DNT settle for less.
1605161130229485569 2022-12-20 12:20:10 +0100 <Esterify_0H> THERE IS NO STOPPING @PeterObi !!! ASO ROCK IS ASSURED!  If you can read the room, you'll know that the ship has long sailed. #Obidients did this!  #ObiDattiInUyo #ObiDattiCampaignInAkwaIbom #PeterObi4President2023 #PeterObiInAkwaIbom  https://t.co/erbR0r7LrI
1605161086830776320 2022-12-20 12:20:00 +0100 <victore38135415> @BayoAdedosu He say make we LP and Obidients be ready, say APC ready in all strata naim talk o, make we no misunderstand him.  He no say APC go win o,but APC too dey desperate simple.  Again he no talk who go win too, make we listen well, na warning he they give us.
1605161039003271168 2022-12-20 12:

1605157876200140800 2022-12-20 12:07:14 +0100 <Atorremy> @JamalPacman Local, loll
1605157872198946816 2022-12-20 12:07:13 +0100 <Chukwuebuka0092> Al mustapha he’s an Asset #Obidatti needs most, some loop holes needs to be filled, then rebuilding of new Nigeria 🇳🇬 #ObiDatti2023 #ObiDattiInUyo #Obidients  https://t.co/05iRk4gPIM
1605157565071085572 2022-12-20 12:06:00 +0100 <maxymaxy4c> @wemimospot I think he should not wait for anyone to ask him to step aside. Our mantra as OBIdients is transparent honesty. Having an ex-convict on the saddle is disquieting
1605157406761254913 2022-12-20 12:05:22 +0100 <igbo_star> All the Obidients in my mentions defending the fulani terrorist herdsmen massacre in Enugu will also face the same fate. I thought they were just naive but I can see they are just evil and won't go unpunished. Nigeria must be dismantled and no amount of blackmail can stop that.  https://t.co/QtMh6uYPia
1605157398859186177 2022-12-20 12:05:20 +0100 <IkediChima> @MEguonho @renoom

1605151447900905472 2022-12-20 11:41:42 +0100 <igweindahouz> Doyin Okupe to pay N13,000,000 fine and bear the tag "ex-convict" for life. He will be removed as #Obidatti023 CDG. We OBIdients have accepted it. Can Tinubu and his party forget about 2023 as we can't afford an ex-convict leading Nigeria? @PeterObi is on the right track!
1605151271941713921 2022-12-20 11:41:00 +0100 <decency_nna> SIGN OF HOPE   Cheerful and smiling faces of #Obidients A'Ibom youths in the presence of H.E #PO during an interactive Town Hall meeting,at Akwa Ibom,today.  Give us,O'Lord Peter Obi,we pray 🙏  https://t.co/ice7W4tIDa
1605151178723332096 2022-12-20 11:40:37 +0100 <Professor_Well> @PeterObi @Professor_Well  Obidients let's follow each other
1605150677528895488 2022-12-20 11:38:38 +0100 <Great_magnus> Dear Obidients don't be distracted the tax ahead is enormous. Alot of distractions, hurdles and bottle necks u will meet on ur road to success. In a basket full of fish some are not worth selling, wat u 

1605144462774108163 2022-12-20 11:13:56 +0100 <NwokieT> @OladejoBilewu Oga, you need Peter Obi more than Peter Obi needs you (you must understand this). If you like don't vote, when the rain starts pouring, it will bath everyone. True Obidients (like myself) have casted their votes for Obi long time ago and more will vote...so just dey play oo
1605144204237500416 2022-12-20 11:12:55 +0100 <stevetom788> I will advise the world and most especially the Obidients to carefully go through these threads, atleast it will help you to understand the Doyin Okupe's case. He was never found guilty of any corrupt acts but for collecting cash above 5m. Thanks  https://t.co/CHGimxZnnl
1605143509777932288 2022-12-20 11:10:09 +0100 <Scion_Son> @OyesileJohn egbon u no try for obidients oo
1605143237039198209 2022-12-20 11:09:04 +0100 <udoka_uka> @MoJayUchenna @PeterObi Ask your fellow OBIDIENTS,
1605143223709704194 2022-12-20 11:09:01 +0100 <Winter_Six> @lonislambert @PeterObi I'm not defending him of co

...

1605169261286404097 2022-12-20 12:52:29 +0100 <aleahshiii> bat nasa who to follow ko yan
1605169260409626624 2022-12-20 12:52:28 +0100 <MaHtAbA29913981> @HamidMirPAK Is bat per descission kro ka Pakistan ka fucture kesa ho ga
1605169257029197824 2022-12-20 12:52:28 +0100 <kythe_bri> BWHHAAHHAHA OSCAR BAT MO NAMAN GINANON YUNG KITTEN😭😭😭😭
1605169252771692544 2022-12-20 12:52:27 +0100 <kmuisy> @bat_thatbitch دمعت تجنننوووون
1605169243506581504 2022-12-20 12:52:24 +0100 <Jurassic_Yoshi> @UneFanDeMSP oe on se bat avec d poney
1605169240885035009 2022-12-20 12:52:24 +0100 <lindependant> #Noel: "Ce qu'on veut, c'est voir un petit garçon avec un porte-bébé dans un catalogue", l'Espagne se bat pour des jouets non-genrés  https://t.co/ipseTBoKHx via @lindependant
1605169232266002433 2022-12-20 12:52:22 +0100 <Saqibkh97418828> @iqrarulhassan Ye bat agar tere baji boley to soch sakta ha
1605169218730618880 2022-12-20 12:52:19 +0100 <flwrs4liu> @_lemonjiu hi 🥺 bat ka sinaktan
1605169218235912192 20

1605168837485363202 2022-12-20 12:50:48 +0100 <haruno_o_kotama> 多くの仕事を自動化するシェル・スクリプト！  https://t.co/DnlJa826T8  boot.bat や boot.vbs や boot.jsなどにpython書類などの 場所を少し記述し、スタートアップフォルダに入れたり、 タイマーソフトで起動日時を指定するだけ！
1605168837170978816 2022-12-20 12:50:48 +0100 <wixyle> Pero bat binalikan?? Pag kasi inggit pikit nalang tayo
1605168833899397120 2022-12-20 12:50:47 +0100 <ecvolusi> @moonareas lucu bats
1605168828488376320 2022-12-20 12:50:45 +0100 <xrevien> BAT WALA AKO PERO NAKA TAG???!?!!?!!
1605168815079190529 2022-12-20 12:50:42 +0100 <findinghannie> @jemirahours @goingtosebongs @kwonulvrr lord hesos mahabagin bat nyo ko binigyan ng 2 pang batang aalagaan 🥹 akala q tapos na kay nae at mikmik
1605168798407217155 2022-12-20 12:50:38 +0100 <mintyscoups> ANONG BSS COMEBACK HUHUHU true ba. bat andaming pasabog. mistape pa ni vernon 🤩
1605168793277415424 2022-12-20 12:50:37 +0100 <Psychopath626> Stokes almost hit the boundary  with his bat 💯😂 #ENGvsPAK #PAKvsENG #PakvsEngland #PAK2022 #CricketTwitter 

1605171460229709824 2022-12-20 13:01:13 +0100 <AjaniOluwole9> @renoomokri Come back to Nigerian an campaign for @atiku. You must win your polling unit ward and local government noisemaker
1605171455263903744 2022-12-20 13:01:12 +0100 <PrinceElvisOse> Nigeria’s Time To Recover With @atiku   #RecoverNigeria #RecoverKatsinaWithAtiku #VotePDP
1605171441049427968 2022-12-20 13:01:08 +0100 <TheParadiseNg> Atiku’s visit to Anambra in order – Traditional rulers council  https://t.co/rmt6zhhsZe
1605171437517537281 2022-12-20 13:01:08 +0100 <miss_my_pals> @EkweThompson @BEjimonye @tjay_256 @FredrickNwabufo And your obi also is in their midst. He is also part of the birds and at least he acknowledged Atiku the thief to be his boss. So he too is a thief, Okupe is a thief.  Also stop pushing a thief on us too
1605171419725594625 2022-12-20 13:01:03 +0100 <oyongha_ojong> @atiku  https://t.co/aLDW2K8rl0
1605171418299236352 2022-12-20 13:01:03 +0100 <Mohd_Aliiiy> @jessetheoracle_ @OlatunjiAyegbu2 @Rea

1605169262825713665 2022-12-20 12:52:29 +0100 <OdelekeSA> The reason @atiku is promising them presidency in the future is because 2nd Niger Bridge has left the realms of promises but now a reality under @MBuhari. PDP scamming since 1807.
1605169261445611520 2022-12-20 12:52:29 +0100 <WizzySpeaks> @jrnaib2 😂 😂 😂 😂 😂 😂 😂 😂 I laugh you and Atiku
1605169235554074626 2022-12-20 12:52:23 +0100 <viasgris> @babalolareal @Oluwasantus @U_Rochas Desperate because he is trying for the first time to contest as president? Can u tell us how many times Atiku has contested? Your answer would be enough definition for "Desperate" 😎😎😎😎 nor come d yarn nonsense here..
1605169217296568320 2022-12-20 12:52:18 +0100 <inzran> @_Khadejah_atiku @_queen_Areefah @FatiWasha4 @hapsaatu @Miss__Meerah @Safnah_Qasim @summeryer_t @Safnarh2 @Usman__Bawa @washafatee @___Hameedah___ @deezzzat habib
1605169211508236288 2022-12-20 12:52:17 +0100 <lonislambert> @IAOkowa Nigeria sai Atiku
1605169207997616131 2022-12-20 12:52:1

1605167423489032192 2022-12-20 12:45:10 +0100 <MohammedRigasa> Welcome to @atiku  Katsina Stete  https://t.co/whZVeB6iVl
1605167416580923392 2022-12-20 12:45:09 +0100 <abduldanja87> Breaking News   Katsina Karkkanda Stadium more than half of it Empty as @atiku @OfficialPDPNig Presidential Candidate visit Katsina  Source #Dikkowatch  https://t.co/CS1yGE2ZY7
1605167383869546497 2022-12-20 12:45:01 +0100 <FemiOyedun1> @MoniGeorge4 Our Message Is Simple And Clear Nigeria, Next President @atiku Is Ready And Prepared To #RecoverNigeria   @OfficialPDPNig is forming the next government.
1605167379260006401 2022-12-20 12:45:00 +0100 <omoh_naija> Mutanen Katsina are ready to receive team #RecoverNigeria. Sannu Da Zuwa Atiku and Okowa 🤝
1605167336427757569 2022-12-20 12:44:50 +0100 <Wynwonder> @harrydaniyan @olusogaowoeye @oluwatosin_3268 3) This has never been about a debate not being his Forte. In what world is Buhari or Atiku a better debater than Tinubu, a first class university graduate? Wha

In [ ]:
pd.set_option('display.max_rows', 10000)

In [ ]:
pd.set_option('display.max_columns', 10000)

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df_all.head(1)

## Making a Copy of the Dataset

In [ ]:
#making a copy
df = df_all.copy()

## Data Cleaning

In [ ]:
#accessing the twint information
df.info()

In [ ]:
#the description of the twint 
df.describe()

In [ ]:
df.shape

In [ ]:
#droping rows which languange =! english
df["language"].unique()

In [ ]:
#language that contains only english
df = df[df["language"].isin(['en'])]

In [ ]:
df.head(2)

In [ ]:
#the shape of the datset
df.shape

In [ ]:
#dropping unnecessary columns
df = df.drop(['id', 'conversation_id', 'timezone', 'place', 'cashtags', 'language', 'name', 'search', 'hashtags','user_id', 'day', 'hour', 'created_at', 'user_id_str', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet','nreplies', 'nretweets', 'quote_url', 'near', 'geo', 'source',  'username', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],axis = 1)

In [ ]:
#checking null values
df.isnull().sum()


In [ ]:
#the get the count of duplicated rows
df_1 = df.duplicated().value_counts()
df_1

In [ ]:
#using regular expression to remove the signs, links, hastags, non_alphabet etc
def clean_text(text):  
    pat1 = r'@[^ ]+'                   #@signs
    pat2 = r'https?://[A-Za-z0-9./]+'  #links
    pat3 = r'\'s'                      #floating s's
    pat4 = r'\#\w+'                     # hashtags
    pat5 = r'&amp '
    pat6 = r'[^A-Za-z\s]'         #remove non-alphabet
    combined_pat = r'|'.join((pat1, pat2,pat3,pat4,pat5, pat6))
    text = re.sub(combined_pat,"",text).lower()
    return text.strip() 

In [ ]:
pat1 = r'@[^ ]+'                   #@signs
pat2 = r'https?://[A-Za-z0-9./]+'  #links
pat3 = r'\'s'                      #floating s's
pat4 = r'\#\w+'                     # hashtags
pat5 = r'&amp '
pat6 = r'[^A-Za-z\s]'         #remove non-alphabet
combined_pat = r'|'.join((pat1, pat2,pat3,pat4,pat5, pat6))

In [ ]:
combined_pat

In [ ]:
#applying the regular expression cleaning
df['tweet'] = df['tweet'].apply(clean_text)

In [ ]:
#keeping it in a new dataframe
df_result = df.drop_duplicates()

In [ ]:
df_result1 = df_result.copy()

In [ ]:
#keeping it in a dataframe
df_result1.head(2)

In [ ]:
print("Running sentiment process")
# creating two new columns(polarity and subjectivity)
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

#applying the subjectivity and polarity function
df_result1["Subjectivity"] = df_result1['tweet'].apply(getSubjectivity)
df_result1["Polarity"] = df_result1['tweet'].apply(getPolarity)

In [ ]:
#seeig a sample of the result
df_result1.sample()

In [ ]:
#function to determine the positive, negative and neutral sentiments
def getAnalysis(text):
    if text < 0:
        return 'negative'
    elif text == 0:
        return 'neutral'
    else:
        return 'positive'
           

In [ ]:
df_result1['Analysis'] = df_result1['Polarity'].apply(getAnalysis)

In [ ]:
df_result1.head(4)

In [ ]:
#locating all Peter_obi's sentiment
Peter_Obi = df_result1.loc[df_result1['Candidate'] == 'Peter Obi']
Peter_Obi.head(5)

In [ ]:
#Peter Obi Analysis Count
AnalysisCount_PeterObi= Peter_Obi['Analysis'].groupby(Peter_Obi['Analysis']).size().reset_index(name='counts')
AnalysisCount_PeterObi

In [ ]:
#locating all BAT's sentiment
BAT = df_result1.loc[df_result1['Candidate'] == 'Bola Ahmed Tinubu']
BAT.head(5)

In [ ]:
#BAT Analysis Count
AnalysisCount_BAT= BAT['Analysis'].groupby(BAT['Analysis']).size().reset_index(name='counts')
AnalysisCount_BAT

In [ ]:
#locating all Atiku's sentiment
Atiku_Abubakar = df_result1.loc[df_result1['Candidate'] == 'Atiku Abubakar']
Atiku_Abubakar.head(5)

In [ ]:
#Atiku Analysis Count
AnalysisCount_AtikuAbubakar= Atiku_Abubakar['Analysis'].groupby(Atiku_Abubakar['Analysis']).size().reset_index(name='counts')
AnalysisCount_AtikuAbubakar

In [ ]:
#grouping all candidates
the_candidates = df_result1.loc[df_result1['Analysis'] == 'positive']
the_candidates.head(5)

In [ ]:
#positive sentiments counts for all candidates
all_candidates = the_candidates['Candidate'].groupby(the_candidates['Candidate']).size().reset_index(name='counts')
all_candidates

In [ ]:
#plotting the results
plt.figure(figsize=(20,6))
sns.barplot(data = all_candidates, x= 'Candidate', y ='counts', palette = 'spring')

#setting the properties for the chart
plt.title('presidential candidates according to their polarity', fontsize = 15 ) 
plt.xlabel('candidates')
plt.ylabel('polarity (positive tweets)')

#saving the chart
plt.show()

In [ ]:
#saving the dataframe to csv
df_result1.to_csv('sentiment.csv')